<a href="https://colab.research.google.com/github/alfcan/CADOCS_NLU_Model/blob/feat%2Fml%2Frequest-translation/request_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation of libraries.

In [ ]:
!pip install torch transformers

In [ ]:
!pip install cmake

In [ ]:
!pip install tokenizers

In [ ]:
!pip install pandas sentencepiece

Import libraries and upload the model and tokenizer.

In [ ]:
import torch
import re
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name_en_it = "Helsinki-NLP/opus-mt-en-it"  # English to Italian Model
model_name_it_en = "Helsinki-NLP/opus-mt-it-en"  # Italian to English Model

tokenizer_en_it = AutoTokenizer.from_pretrained(model_name_en_it)
model_en_it = AutoModelForSeq2SeqLM.from_pretrained(model_name_en_it).to(device)

tokenizer_it_en = AutoTokenizer.from_pretrained(model_name_it_en)
model_it_en = AutoModelForSeq2SeqLM.from_pretrained(model_name_it_en).to(device)

Define the function for requests translation, excluding words: "community", "smells", "report", "repository" and "CADOCS"

In [ ]:
def translate_dataset(dataset_path, output_path, tokenizer, model):
    dataset = pd.read_csv(dataset_path, sep=';')
    translations = []

    for index, row in dataset.iterrows():
        text = row['request']

        text = text.replace("community", "<TOKEN1>")
        text = text.replace("smells", "<TOKEN2>")
        text = text.replace("report", "<TOKEN3>")
        text = text.replace("repository", "<TOKEN4>")
        text = text.replace("CADOCS", "<TOKEN5>")

        inputs = tokenizer.encode(text, return_tensors="pt", padding=True)
        translation = tokenizer.decode(model.generate(inputs.to("cuda")).squeeze(), skip_special_tokens=True)

        translation = translation.replace("<TOKEN1>", "community")
        translation = translation.replace("<TOKEN2>", "smells")
        translation = translation.replace("<TOKEN3>", "report")
        translation = translation.replace("<TOKEN4>", "repository")
        translation = translation.replace("<TOKEN5>", "CADOCS")
        translations.append(translation)
    
    dataset["translation"] = translations
    dataset.to_csv(output_path, index=False)

Dataset translation from english to italian.

In [ ]:
dataset_path_en_it = "path_english_dataset"
output_path_en_it = "output_path"

translate_dataset(dataset_path_en_it, output_path_en_it, tokenizer_en_it, model_en_it)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Dataset translation from italian to english.

In [ ]:
dataset_path_it_en = "path_italian_dataset"
output_path_it_en = "output_path"

translate_dataset(dataset_path_it_en, output_path_it_en, tokenizer_it_en, model_it_en)